<a href="https://colab.research.google.com/github/EudoraHan/CatBoost-Performance-Testing/blob/develop/CPU%26GPU_in_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CatBoost Property Test

### Yun Han 7/30/2019

# 1. Epilson

In [33]:
!pip install catboost

In [0]:
from catboost.datasets import epsilon

train, test = epsilon()

X_train, y_train = train.iloc[:,1:], train[0]
X_test, y_test = test.iloc[:,1:], test[0]


In [35]:
X_train.shape

(400000, 2000)

In [36]:
X_test.shape

(100000, 2000)

In [0]:
!pip install -U -q PyDrive

     |████████████████████████████████| 993kB 9.6MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# Create & upload a text file.
#你想要导出的文件的名字
uploaded = drive.CreateFile({'title': 'OK.csv'})
#改为之前生成文件的名字
uploaded.SetContentFile('over.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1DPAcA9VzfP6HuhhSt0ifk9vXSsdCgobH


In [37]:
X_train.shape


(400000, 2000)

### Training on CPU

In [0]:
from catboost import CatBoostClassifier
import timeit

def train_on_cpu():  
  model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03
  )
  
  model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=10
  );   
      
cpu_time = timeit.timeit('train_on_cpu()', 
                         setup="from __main__ import train_on_cpu", 
                         number=1)

print('Time to fit model on CPU: {} sec'.format(int(cpu_time)))

0:	learn: 0.6878004	test: 0.6878645	best: 0.6878645 (0)	total: 7.54s	remaining: 2h 5m 33s
10:	learn: 0.6459486	test: 0.6467108	best: 0.6467108 (10)	total: 1m 24s	remaining: 2h 6m 29s
20:	learn: 0.6170315	test: 0.6180805	best: 0.6180805 (20)	total: 2m 39s	remaining: 2h 4m
30:	learn: 0.5947894	test: 0.5960342	best: 0.5960342 (30)	total: 3m 52s	remaining: 2h 55s
40:	learn: 0.5766997	test: 0.5780390	best: 0.5780390 (40)	total: 5m 2s	remaining: 1h 58m 5s
50:	learn: 0.5616803	test: 0.5631138	best: 0.5631138 (50)	total: 6m 10s	remaining: 1h 54m 54s
60:	learn: 0.5485252	test: 0.5499754	best: 0.5499754 (60)	total: 7m 19s	remaining: 1h 52m 42s
70:	learn: 0.5371519	test: 0.5385685	best: 0.5385685 (70)	total: 8m 29s	remaining: 1h 51m 6s
80:	learn: 0.5267357	test: 0.5282210	best: 0.5282210 (80)	total: 9m 40s	remaining: 1h 49m 48s
90:	learn: 0.5175479	test: 0.5190800	best: 0.5190800 (90)	total: 10m 52s	remaining: 1h 48m 36s
100:	learn: 0.5092512	test: 0.5108786	best: 0.5108786 (100)	total: 12m 7s	re

### Training on GPU

In [0]:
def train_on_gpu():  
  model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.03,
    task_type='GPU'
  )
  
  model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=10
  );     
      
gpu_time = timeit.timeit('train_on_gpu()', 
                         setup="from __main__ import train_on_gpu", 
                         number=1)

print('Time to fit model on GPU: {} sec'.format(int(gpu_time)))
print('GPU speedup over CPU: ' + '%.2f' % (cpu_time/gpu_time) + 'x')

0:	learn: 0.6877672	test: 0.6878640	best: 0.6878640 (0)	total: 204ms	remaining: 20.2s
10:	learn: 0.6457422	test: 0.6464576	best: 0.6464576 (10)	total: 1.3s	remaining: 10.6s
20:	learn: 0.6163275	test: 0.6174585	best: 0.6174585 (20)	total: 2.36s	remaining: 8.87s
30:	learn: 0.5943049	test: 0.5956978	best: 0.5956978 (30)	total: 3.38s	remaining: 7.52s
40:	learn: 0.5763316	test: 0.5778145	best: 0.5778145 (40)	total: 4.39s	remaining: 6.32s
50:	learn: 0.5607705	test: 0.5623940	best: 0.5623940 (50)	total: 5.42s	remaining: 5.2s
60:	learn: 0.5478194	test: 0.5495075	best: 0.5495075 (60)	total: 6.41s	remaining: 4.09s
70:	learn: 0.5360011	test: 0.5377425	best: 0.5377425 (70)	total: 7.37s	remaining: 3.01s
80:	learn: 0.5258042	test: 0.5275755	best: 0.5275755 (80)	total: 8.37s	remaining: 1.96s
90:	learn: 0.5165436	test: 0.5183847	best: 0.5183847 (90)	total: 9.38s	remaining: 927ms
99:	learn: 0.5089647	test: 0.5108700	best: 0.5108700 (99)	total: 10.2s	remaining: 0us
bestTest = 0.5108699609
bestIteration 

In [0]:
from catboost import CatBoostClassifier
classifier_cat = CatBoostClassifier(iterations = 100, task_type = 'GPU')

In [0]:
# Train the model
classifier_cat.fit(X_train, y_train)

In [0]:
# Prediction of test data
classifier_cat.predict(X_test)

In [0]:
# Accuracy of test data
classifier_cat.score(X_test, y_test)

# 2. Credit card lost

In [17]:
from google.colab import files
uploaded = files.upload()

Saving creditcard lost.csv to creditcard lost.csv


In [0]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['creditcard lost.csv']))
#df.head()

In [20]:
df.head()

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,...,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120
0,1,5.04,5.77,6.04,3.91,0.66,1.04,1.83,5.41,5.45,6.91,6.70,1.17,3.08,3.44,1.83,5.32,5.16,4.62,4.58,3.98,6.57,5.18,1.79,2.99,7.95,21.02,1.70,0.00,4.55,6.90,0.00,0.00,0.00,2.25,3.16,5.85,6.26,5.06,0.00,...,4.94,5.75,6.23,5.80,1.45,2.63,2.19,5.18,8.17,3.39,1.14,7.95,7.95,8.52,3.41,3.98,0.0,3.45,0.00,0.0,3.45,0.00,0.0,0.00,20.69,7.94,7.70,2.97,6.88,4.54,0.68,7.31,8.04,10.73,5.58,0.00,7.52,5.37,6.18,4.24
1,1,5.98,2.90,1.94,1.42,0.10,3.29,0.81,9.09,8.04,4.65,2.65,0.46,2.25,6.63,0.00,6.19,5.06,0.56,4.80,6.55,3.49,0.44,0.00,3.93,10.62,4.28,3.83,3.98,7.52,24.07,9.26,0.00,0.00,0.47,0.04,10.31,9.35,4.45,0.04,...,5.17,7.54,13.27,3.82,0.00,0.34,0.00,10.48,6.55,0.44,0.00,12.83,12.83,5.31,2.51,1.33,0.0,4.63,1.85,0.0,2.78,2.78,0.0,0.00,0.93,12.21,6.96,0.10,4.20,0.88,0.19,10.44,7.17,2.96,6.75,0.17,5.40,8.32,5.40,1.57
2,1,7.04,8.92,3.24,0.01,0.00,4.50,0.00,8.41,6.97,12.65,9.24,0.00,0.54,10.11,0.06,10.43,12.13,1.42,13.91,10.66,3.73,3.05,0.00,2.61,2.86,8.57,10.00,0.00,8.57,11.11,3.70,0.00,3.70,0.00,0.00,9.51,5.19,6.20,0.00,...,7.11,8.28,8.66,5.33,0.00,0.13,0.00,6.01,6.16,0.08,0.00,10.00,7.14,11.43,20.00,2.86,0.0,7.41,14.81,0.0,0.00,0.00,0.0,0.00,11.11,8.47,9.45,0.00,4.60,0.43,0.00,3.77,2.70,6.84,7.28,0.00,5.60,7.52,5.93,3.89
3,1,13.30,7.16,5.69,1.98,0.71,2.20,0.22,13.89,1.75,0.00,3.27,0.00,10.26,9.46,4.42,4.14,4.08,4.32,4.22,4.11,3.83,4.08,4.36,3.99,7.96,6.19,5.31,4.42,11.50,32.14,3.57,0.00,0.00,0.58,0.13,5.83,1.45,11.60,0.00,...,4.11,4.15,4.13,4.20,4.37,4.30,4.54,4.15,4.11,4.28,0.00,9.73,4.42,3.54,7.96,4.42,0.0,7.14,3.57,0.0,0.00,0.00,0.0,0.00,3.57,2.95,7.28,0.00,9.54,5.71,0.48,4.77,1.13,2.67,8.83,0.00,4.09,4.14,4.16,4.23
4,1,5.24,5.09,5.31,1.05,0.00,4.24,0.34,5.25,5.59,9.58,3.32,0.00,5.85,10.89,2.16,6.13,5.88,4.22,5.15,5.54,5.85,4.68,1.15,1.54,9.42,7.48,2.77,3.88,6.65,13.64,3.03,4.55,4.55,0.24,0.08,8.67,8.11,4.08,0.00,...,5.93,6.01,6.05,6.07,1.64,3.17,2.25,6.44,6.60,2.91,0.28,9.14,9.42,4.99,3.60,2.49,0.0,15.15,1.52,0.0,0.00,0.00,0.0,4.55,9.09,7.24,7.63,0.00,5.11,3.71,0.00,6.01,4.16,4.49,6.14,0.00,6.18,6.10,6.06,5.86


In [24]:
X

array([[5.04, 5.77, 6.04, ..., 5.37, 6.18, 4.24],
       [5.98, 2.9 , 1.94, ..., 8.32, 5.4 , 1.57],
       [7.04, 8.92, 3.24, ..., 7.52, 5.93, 3.89],
       ...,
       [2.74, 5.42, 8.79, ..., 6.75, 4.98, 5.21],
       [7.51, 6.76, 4.73, ..., 4.94, 5.  , 4.93],
       [3.81, 6.79, 7.66, ..., 5.17, 5.06, 4.72]])

In [0]:
# Get X and y
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [25]:
# Split data into training and testing
from sklearn import model_selection

# Reserve 20% for testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

print('Training data has %d observation with %d features' % X_train.shape)
print('Test data has %d observation with %d features' % X_test.shape)

Training data has 12000 observation with 120 features
Test data has 3000 observation with 120 features


In [26]:
!pip install catboost
from catboost import CatBoostClassifier
classifier_cat = CatBoostClassifier(iterations = 100, task_type = 'GPU')

In [28]:
X_train.shape


(12000, 120)

In [29]:
import timeit
def train_on_cpu():  
  model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03
  )
  
  model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=100
  );   
      
cpu_time = timeit.timeit('train_on_cpu()', 
                         setup="from __main__ import train_on_cpu", 
                         number=1)

print('Time to fit model on CPU: {} sec'.format(int(cpu_time)))

0:	learn: 0.6678724	test: 0.6675403	best: 0.6675403 (0)	total: 264ms	remaining: 4m 24s
100:	learn: 0.3198527	test: 0.3176252	best: 0.3176252 (100)	total: 26.4s	remaining: 3m 54s
200:	learn: 0.3068348	test: 0.3165573	best: 0.3164839 (196)	total: 51.4s	remaining: 3m 24s
300:	learn: 0.2974008	test: 0.3169241	best: 0.3163827 (208)	total: 1m 15s	remaining: 2m 56s
400:	learn: 0.2874031	test: 0.3172878	best: 0.3163827 (208)	total: 1m 40s	remaining: 2m 29s
500:	learn: 0.2786824	test: 0.3177666	best: 0.3163827 (208)	total: 2m 4s	remaining: 2m 3s
600:	learn: 0.2700714	test: 0.3180958	best: 0.3163827 (208)	total: 2m 28s	remaining: 1m 38s
700:	learn: 0.2630478	test: 0.3188492	best: 0.3163827 (208)	total: 2m 52s	remaining: 1m 13s
800:	learn: 0.2564199	test: 0.3192582	best: 0.3163827 (208)	total: 3m 15s	remaining: 48.7s
900:	learn: 0.2508234	test: 0.3196992	best: 0.3163827 (208)	total: 3m 39s	remaining: 24.1s
999:	learn: 0.2447655	test: 0.3200633	best: 0.3163827 (208)	total: 4m 3s	remaining: 0us

be

In [30]:
def train_on_gpu():  
  model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    task_type='GPU'
  )
  
  model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=100
  );     
      
gpu_time = timeit.timeit('train_on_gpu()', 
                         setup="from __main__ import train_on_gpu", 
                         number=1)

print('Time to fit model on GPU: {} sec'.format(int(gpu_time)))
print('GPU speedup over CPU: ' + '%.2f' % (cpu_time/gpu_time) + 'x')

0:	learn: 0.6678815	test: 0.6674659	best: 0.6674659 (0)	total: 75.3ms	remaining: 1m 15s
100:	learn: 0.3164914	test: 0.3169017	best: 0.3169017 (100)	total: 4.13s	remaining: 36.8s
200:	learn: 0.3025696	test: 0.3158918	best: 0.3157258 (180)	total: 8.06s	remaining: 32s
300:	learn: 0.2915746	test: 0.3155904	best: 0.3155712 (299)	total: 12s	remaining: 27.8s
400:	learn: 0.2813490	test: 0.3159931	best: 0.3155262 (377)	total: 15.9s	remaining: 23.8s
500:	learn: 0.2709261	test: 0.3169734	best: 0.3155262 (377)	total: 19.9s	remaining: 19.8s
600:	learn: 0.2622394	test: 0.3172880	best: 0.3155262 (377)	total: 23.8s	remaining: 15.8s
700:	learn: 0.2531303	test: 0.3174485	best: 0.3155262 (377)	total: 27.8s	remaining: 11.8s
800:	learn: 0.2452772	test: 0.3179960	best: 0.3155262 (377)	total: 31.8s	remaining: 7.89s
900:	learn: 0.2378677	test: 0.3185316	best: 0.3155262 (377)	total: 35.7s	remaining: 3.92s
999:	learn: 0.2316635	test: 0.3188425	best: 0.3155262 (377)	total: 39.7s	remaining: 0us
bestTest = 0.31552

In [0]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'learning_rate': 0.03
}


In [32]:
# Use time function to measure time elapsed
import time
start = time.time()


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_train)

end = time.time()
print(end - start)

[1]	training's binary_logloss: 0.332892
[2]	training's binary_logloss: 0.331325
[3]	training's binary_logloss: 0.329847
[4]	training's binary_logloss: 0.328469
[5]	training's binary_logloss: 0.326981
[6]	training's binary_logloss: 0.325567
[7]	training's binary_logloss: 0.324211
[8]	training's binary_logloss: 0.32296
[9]	training's binary_logloss: 0.321699
[10]	training's binary_logloss: 0.320385
[11]	training's binary_logloss: 0.319137
[12]	training's binary_logloss: 0.317881
[13]	training's binary_logloss: 0.316514
[14]	training's binary_logloss: 0.315302
[15]	training's binary_logloss: 0.314045
[16]	training's binary_logloss: 0.312827
[17]	training's binary_logloss: 0.3116
[18]	training's binary_logloss: 0.310397
[19]	training's binary_logloss: 0.309276
[20]	training's binary_logloss: 0.308279
[21]	training's binary_logloss: 0.30713
[22]	training's binary_logloss: 0.30598
[23]	training's binary_logloss: 0.30497
[24]	training's binary_logloss: 0.303916
[25]	training's binary_logloss:

# 3. Customer Satisfaction

In [0]:
from google.colab import files
uploaded = files.upload()

Saving customer_satisfaction.csv to customer_satisfaction.csv


In [0]:
import io
import pandas as pd

data = pd.read_csv(io.BytesIO(uploaded['customer_satisfaction.csv']))
data.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170000,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030000,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.00,2.07,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770000,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91.56,138.84,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970000,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,40501.08,13501.47,0.0,0.0,0.0,0.0,0.0,0.0,85501.89,85501.89,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0


In [0]:
# Get X and y
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [0]:
# Split data into training and testing
from sklearn import model_selection

# Reserve 20% for testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)

print('Training data has %d observation with %d features' % X_train.shape)
print('Test data has %d observation with %d features' % X_test.shape)

Training data has 60816 observation with 370 features
Test data has 15204 observation with 370 features


## 3.1 CatBoost

### Train in CPU

In [0]:
from catboost import CatBoostClassifier
classifier_cat = CatBoostClassifier(iterations = 100, task_type = 'GPU')

In [0]:
import timeit
def train_on_cpu():  
  model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03
  )
  
  model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=100
  );   
      
cpu_time = timeit.timeit('train_on_cpu()', 
                         setup="from __main__ import train_on_cpu", 
                         number=1)

print('Time to fit model on CPU: {} sec'.format(int(cpu_time)))

0:	learn: 0.6499851	test: 0.6499793	best: 0.6499793 (0)	total: 195ms	remaining: 3m 14s
100:	learn: 0.1366798	test: 0.1390385	best: 0.1390385 (100)	total: 11s	remaining: 1m 38s
200:	learn: 0.1323332	test: 0.1365640	best: 0.1365614 (196)	total: 21.5s	remaining: 1m 25s
300:	learn: 0.1304998	test: 0.1360108	best: 0.1360085 (299)	total: 31.4s	remaining: 1m 12s
400:	learn: 0.1289568	test: 0.1356632	best: 0.1356632 (400)	total: 41.4s	remaining: 1m 1s
500:	learn: 0.1274309	test: 0.1355078	best: 0.1355078 (500)	total: 51.5s	remaining: 51.3s
600:	learn: 0.1260395	test: 0.1354352	best: 0.1354352 (600)	total: 1m 1s	remaining: 41.1s
700:	learn: 0.1248485	test: 0.1353525	best: 0.1353525 (700)	total: 1m 12s	remaining: 30.8s
800:	learn: 0.1236035	test: 0.1353367	best: 0.1353167 (781)	total: 1m 23s	remaining: 20.6s
900:	learn: 0.1224571	test: 0.1352741	best: 0.1352424 (876)	total: 1m 33s	remaining: 10.3s
999:	learn: 0.1212619	test: 0.1352166	best: 0.1351861 (984)	total: 1m 44s	remaining: 0us

bestTest 

### Train in GPU

In [0]:
def train_on_gpu():  
  model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    task_type='GPU'
  )
  
  model.fit(
      X_train, y_train,
      eval_set=(X_test, y_test),
      verbose=100
  );     
      
gpu_time = timeit.timeit('train_on_gpu()', 
                         setup="from __main__ import train_on_gpu", 
                         number=1)

print('Time to fit model on GPU: {} sec'.format(int(gpu_time)))
print('GPU speedup over CPU: ' + '%.2f' % (cpu_time/gpu_time) + 'x')

0:	learn: 0.6419186	test: 0.6419137	best: 0.6419137 (0)	total: 46ms	remaining: 45.9s
100:	learn: 0.1360534	test: 0.1385616	best: 0.1385616 (100)	total: 1.25s	remaining: 11.1s
200:	learn: 0.1318218	test: 0.1362891	best: 0.1362869 (199)	total: 2.32s	remaining: 9.22s
300:	learn: 0.1297227	test: 0.1357183	best: 0.1357183 (300)	total: 3.38s	remaining: 7.84s
400:	learn: 0.1282696	test: 0.1354960	best: 0.1354960 (400)	total: 4.44s	remaining: 6.63s
500:	learn: 0.1267707	test: 0.1354005	best: 0.1353966 (494)	total: 5.51s	remaining: 5.49s
600:	learn: 0.1253700	test: 0.1352449	best: 0.1352449 (600)	total: 6.62s	remaining: 4.39s
700:	learn: 0.1239492	test: 0.1352796	best: 0.1352240 (625)	total: 7.78s	remaining: 3.32s
800:	learn: 0.1228055	test: 0.1352042	best: 0.1351763 (784)	total: 8.91s	remaining: 2.21s
900:	learn: 0.1215164	test: 0.1351012	best: 0.1351012 (900)	total: 10s	remaining: 1.1s
999:	learn: 0.1203040	test: 0.1350780	best: 0.1350476 (973)	total: 11.1s	remaining: 0us
bestTest = 0.1350476

## 3.2 LGBM

### Train in LGBM

In [0]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'learning_rate': 0.03
}


In [0]:
# Use time function to measure time elapsed
import time
start = time.time()


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=lgb_train)

end = time.time()
print(end - start)

[1]	training's binary_logloss: 0.163677
[2]	training's binary_logloss: 0.1612
[3]	training's binary_logloss: 0.159019
[4]	training's binary_logloss: 0.157068
[5]	training's binary_logloss: 0.155332
[6]	training's binary_logloss: 0.153751
[7]	training's binary_logloss: 0.152293
[8]	training's binary_logloss: 0.150965
[9]	training's binary_logloss: 0.149755
[10]	training's binary_logloss: 0.148618
[11]	training's binary_logloss: 0.147578
[12]	training's binary_logloss: 0.146561
[13]	training's binary_logloss: 0.145607
[14]	training's binary_logloss: 0.144724
[15]	training's binary_logloss: 0.143862
[16]	training's binary_logloss: 0.143089
[17]	training's binary_logloss: 0.14235
[18]	training's binary_logloss: 0.141639
[19]	training's binary_logloss: 0.140965
[20]	training's binary_logloss: 0.140324
[21]	training's binary_logloss: 0.139712
[22]	training's binary_logloss: 0.139141
[23]	training's binary_logloss: 0.138581
[24]	training's binary_logloss: 0.138023
[25]	training's binary_loglo